In [15]:
from psycopg import connect
from uuid import UUID
from closure import clsr_insert, Inode

In [35]:
DB_HOST = "localhost"  # Use 'localhost' or the Docker network alias if running in a Docker network
DB_PORT = "5432"  # Default PostgreSQL port
DB_NAME = "postgres"  # Name of the database
DB_USER = "postgres"  # PostgreSQL user
DB_PASSWORD = "password"  # PostgreSQL password
conn = connect(
    host=DB_HOST,
    port=DB_PORT,
    # database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
)

In [36]:
cur = conn.cursor()

In [13]:
def make_node(i: int, name: str = "NODE") -> Inode:
    return Inode(id=None, name=f"{name}{i}", template=None, node_type="node")

n=20
nodes = [make_node(i) for i in range(n)]

In [16]:

owner = UUID("3c07ee61-4fc0-44ca-b2ad-e6820f614f74")
(id0,) = clsr_insert(cur, owner, None, nodes[0])
(id1,) = clsr_insert(cur, owner, id0, nodes[1])
(id2,) = clsr_insert(cur, owner, id0, nodes[2])
(id3,) = clsr_insert(cur, owner, id1, nodes[3])
(id4,) = clsr_insert(cur, owner, id1, nodes[4])
(id5,) = clsr_insert(cur, owner, id1, nodes[5])
(id6,) = clsr_insert(cur, owner, id2, nodes[6])
(id7,) = clsr_insert(cur, owner, id2, nodes[7])
(id8,) = clsr_insert(cur, owner, id2, nodes[8])
(id9,) = clsr_insert(cur, owner, id2, nodes[9])
(id10,) = clsr_insert(cur, owner, id4, nodes[10])
(id11,) = clsr_insert(cur, owner, id6, nodes[11])
(id12,) = clsr_insert(cur, owner, id8, nodes[12])
(id13,) = clsr_insert(cur, owner, id8, nodes[13])
(id14,) = clsr_insert(cur, owner, id8, nodes[14])
(id15,) = clsr_insert(cur, owner, id4, nodes[15])
(id16,) = clsr_insert(cur, owner, id10, nodes[16])
(id17,) = clsr_insert(cur, owner, id10, nodes[17])
(id18,) = clsr_insert(cur, owner, id16, nodes[18])
(id19,) = clsr_insert(cur, owner, id16, nodes[19])

In [37]:
cur.execute("""SELECT select_child_path(%s, %s)""",(id2, owner),)
rows = cur.fetchall()

for r in rows:
    print(r)

cur.execute(
    """
        SELECT select_internal_child_path(%s, %s)""",
    (id2, owner),
)
rows = cur.fetchall()
for r in rows:
    print(r)


RaiseException: ID f7e207db-5872-488f-bab9-7440500abee6, is not owned by 3c07ee61-4fc0-44ca-b2ad-e6820f614f74
CONTEXT:  PL/pgSQL function is_owner(uuid,uuid) line 13 at RAISE
SQL statement "CALL is_owner(p_parent_id, p_owner)"
PL/pgSQL function select_child_path(uuid,uuid) line 4 at CALL

In [23]:
qoriginal = """    WITH child_paths AS (
        SELECT child FROM select_internal_child_path(%s, %s)
    )
    SELECT 
    DISTINCT 
        n.id, n.name, 
    string_agg(n.name, '.') OVER (PARTITION BY t.child ORDER BY t.depth DESC) AS path, 
        n.template, n.node_type
    FROM inode n
    JOIN link t ON t.parent = n.id
    WHERE t.child IN (SELECT * FROM child_paths cp)
    ;"""

In [30]:
q = """    WITH child_paths AS (
        SELECT child FROM select_internal_child_path(%s, %s)
    )
    SELECT 
        n.id, t.child,n.name, t.depth
    FROM inode n
    JOIN link t ON t.parent = n.id
    WHERE t.child IN (SELECT * FROM child_paths cp)
    ;"""
    
cur.execute(q, (id2, owner))
rows = cur.fetchall()
for r in rows:
    print(r)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block

In [31]:
conn.close()